# Build a shower environment to get the temperature right every time!

# import dependencies

In [46]:
# import GYM stuff
import gym
from gym import Env
from gym.spaces import Discrete, Box, Tuple, Dict, MultiBinary, MultiDiscrete

# import helpers
import numpy as np
import random
import os

# import stable_baselines stuff
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# Types of spaces

In [47]:
Discrete(4).sample()   #Discerete numbers between 0 and 3

1

In [48]:
Box(0,1,shape=(3,3)).sample()   #Box(lower_value, Upper_value, shape_of_the_output, type_the_values)

array([[0.28563455, 0.7384383 , 0.00556086],
       [0.22023042, 0.02159934, 0.6109321 ],
       [0.7388274 , 0.7118428 , 0.11690365]], dtype=float32)

In [49]:
Box(0,2,shape=(4,)).sample()

array([0.681871 , 1.6813501, 1.0623902, 1.3560464], dtype=float32)

In [50]:
Tuple((Discrete(4), Box(0,1,shape=(3,3)))).sample()

(1, array([[0.43443868, 0.06814898, 0.99486184],
        [0.4998992 , 0.20227282, 0.8274358 ],
        [0.7684183 , 0.58427995, 0.53814536]], dtype=float32))

In [51]:
Dict({'height':Discrete(3), 'speed':Box(1,2,shape=(2,2))}).sample()

OrderedDict([('height', 1), ('speed', array([[1.8572938, 1.6744769],
                     [1.8414911, 1.7709492]], dtype=float32))])

In [52]:
MultiBinary(4).sample()

array([0, 0, 1, 0], dtype=int8)

In [53]:
MultiDiscrete([2,5,8]).sample()  #Return an arraycontains 3 values between:  [0 and 1, 0 and 4 , 0 and 7]

array([0, 4, 7], dtype=int64)

# Building an environment 
- Build an agent to give us the best shower possible
- Randomly temperature
- Between 37 and 39

In [55]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)         # we have 3 different actions : 0=decreasing by-1, 1=Unchanged , 2=increasing by+1
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))    # equals to : Box(0,100, shape=(1,)) => Return One value between 0 and 100
        self.state = 38 + random.randint(-3,3)
        self.shower_length = 60
       
    def step(self, action):
        #apply temp adj
        self.state += action-1
        
        #Decrease shower time
        self.shower_length -= 1
        
        #Calculate reward
        if self.state>=37 and self.state<=39:
            reward = 1
        else:
            reward = -1
        
        if self.shower_length<=0:
            done = True
        else:
            done = False
        
        info = {}
        
        return self.state, reward, done, info
    
    def render(self):
#         implement viz
        pass

    def reset(self):
        self.state = np.array([38+random.randint(-3,3)]).astype(float)
        self.shower_length = 60
        return self.state
    

In [57]:
env = ShowerEnv()

In [58]:
env.action_space

Discrete(3)

In [60]:
env.observation_space.sample()

array([38.0714], dtype=float32)

# Test Environment

In [61]:
# To deploy the model, we have to observe the environment and take action based on observation, not only sample action. => change action to observation
episodes = 5
for episode in range (1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-2
Episode:2 Score:-42
Episode:3 Score:12
Episode:4 Score:-44
Episode:5 Score:-16


# Train model

In [63]:
log_path = os.path.join('Training','Logs')
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

C:\Users\leilaa\Anaconda3\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 8000). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:115.)
  return torch._C._cuda_getDeviceCount() > 0


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [64]:
model.learn(total_timesteps=1000)

Logging to Training\Logs\PPO_4
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -29.8    |
| time/              |          |
|    fps             | 70       |
|    iterations      | 1        |
|    time_elapsed    | 28       |
|    total_timesteps | 2048     |
---------------------------------


# Save model

In [66]:
shower_path = os.path.join('Training','Saved Models', 'Shower_Model_PPO')

In [67]:
model.save(shower_path)

In [68]:
del model

In [69]:
model = PPO.load(shower_path, env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [70]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)
# Return (mean reward , standard deviation)

C:\Users\leilaa\Anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


(-36.0, 48.0)